In [25]:
import requests
import re

MISTRAL_API_KEY = "e6ojRsfwNwYFzpVQjpHtX8YavnOrIjcI"
MODEL = "mistral-small-latest"
API_URL = "https://api.mistral.ai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

def check_credibility(text):
    prompt = f"""
You are a misinformation detection expert.

Analyze the following online content for credibility:
\"\"\"{text}\"\"\"

Step 1: Identify if the text contains misleading, exaggerated, or false claims.
Step 2: Detect patterns typical of misinformation.
Step 3: Output a credibility score from 0 (false) to 100 (credible) and a reason.

Respond in this format only:
Credibility Score: <score>
Reason: <brief reason>
"""
    body = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post(API_URL, headers=headers, json=body)
    if response.status_code != 200:
        raise Exception(f"❌ API Error: {response.status_code} - {response.text}")

    content = response.json()['choices'][0]['message']['content']
    match = re.search(r"Credibility Score: (\d+)\s*Reason: (.*)", content, re.DOTALL)
    
    if match:
        score = int(match.group(1))
        reason = match.group(2).strip()
        return score, reason
    else:
        return None, "Couldn't parse response."

# === TEST ===
if __name__ == "__main__":
    text = "hannah montana is a real person"
    try:
        score, reason = check_credibility(text)
        print(f"\n🔎 Score: {score}/100")
        print(f"📝 Reason: {reason}")
    except Exception as e:
        print(e)



🔎 Score: 0/100
📝 Reason: "Hannah Montana" is a fictional character from the Disney Channel television series of the same name, portrayed by actress Miley Cyrus. The claim that "Hannah Montana is a real person" is false.
